# 题目: 28. 找出字符串中第一个匹配的下标
* 给你两个字符串 haystack 和 needle ，请你在 haystack 字符串中找出 needle 字符串的第一个匹配项的下标（下标从 0 开始）。如果 needle 不是 haystack 的一部分，则返回  -1 。

 

* 示例 1：

    * 输入：haystack = "sadbutsad", needle = "sad"
    * 输出：0
    * 解释："sad" 在下标 0 和 6 处匹配。
    * 第一个匹配项的下标是 0 ，所以返回 0 。

* 示例 2：

    * 输入：haystack = "leetcode", needle = "leeto"
    * 输出：-1
    * 解释："leeto" 没有在 "leetcode" 中出现，所以返回 -1 。
 

* 提示：

* 1 <= haystack.length, needle.length <= $10^4$
* haystack 和 needle 仅由小写英文字符组成

来源：力扣（LeetCode）
链接：https://leetcode.cn/problems/find-the-index-of-the-first-occurrence-in-a-string
著作权归领扣网络所有。商业转载请联系官方授权，非商业转载请注明出处。

In [1]:
# 测试: 当needle 为空时返回什么? 答案是0, 这是C语言 strstr() 定义的, Python本身底层也是C/C++, 满足这个定义
"" in "a"

True

* 分析:
    * 本题是KMP算法的经典问题, 具体理论查看KMP算法的笔记文件

* 解题思路 :
1. 构建 Next 数组 (next数组 等价于 前缀表记录的数据 "统一减1")
    * 主要解决三个问题:
        * 初始化
        * 前缀与后缀不相同的情况
        * 前缀与后缀相同的情况 
2. 利用 next 数组做匹配

## 对**模板串**(needle)构建 next 数组

In [2]:
def get_next_arr(needle):
    '''
    关键思路: 使用两个指针 i,k, i指向后缀的末尾, k指向前缀的末尾(同时也是最长相等前后缀的长度)
    注意: 
        1. next[index] 表示的是 index(包含index) 字符之前的 子串的 最长相等前后缀的长度 
        2. next 数组最小值是-1, 当 相同前后缀的最大长度为0时, next 数组记录的是-1 
        3. 假设needle最少得有1个元素, needle 为空字符串的情况不应生成 next 数组
    '''
    # 初始化:
    next_arr = ['' for _ in range(len(needle))]
    k = -1                              # 当 needle 只有1个元素时, 前后缀都不存在, 此时next数组应该记录为 -1
    next_arr[0] = k                    
    
    #  遍历模板串 needle, 逐步构建next数组
    for i in range(1, len(needle)):     # 注意 i 从 1 开始遍历
        
        # 处理前缀与后缀不相同的情况.  (注意: 对于当前遍历到的每一个子串, 都是从最长的前缀开始进行判断, 不相等时就缩短对应的前缀)
        # 做完这一步, 前缀末尾指针 k 要么会停在前后缀相等的位置, 要么直接为-1
        while (k > -1 and needle[i] != needle[k+1]):
            k = next_arr[k]                 # 这里按照 next 数组的定义去移动k: next[index] 表示的是 index(包含index) 字符之前的 子串的 最长相等前后缀的长度    ==〉 这里也是我们写的 【循环不变量】
                                        # 前缀被缩短到前一个字符里存储的最长相等子串的位置, 这个位置在 next 数组里保存着, 由于 i 是从 needle 头部开始遍历, 所以无论如何next数组都是能找到对应值的
    
        # 处理 前后缀相等的情况.  注意: 前面的 while 走完, k 要么为-1, 要么走到前后缀相等的位置
        if needle[i] == needle[k+1]:
            k += 1                      # 如果找到了前后缀相等的位置, 那么就要把 k+1 （因为k不仅表示前缀的末尾，还表示着最长相等前后缀的长度，现在找到了更长的像等前后缀，当然要加1）, k+1后还相当于方便了下一次遍历
                                        # i 的增加在 for 循环里完成

        next_arr[i] = k 
    return next_arr

### 生成前缀表过程 动图:
![jupyter](https://code-thinking.cdn.bcebos.com/gifs/KMP%E7%B2%BE%E8%AE%B23.gif)


In [4]:
# 测试一下 next 数组生成函数
needle = "aabaaf"
# needle = "abac"
temp = get_next_arr(needle)
print(f'Next array: {temp}')

import numpy as np 
print(f'prefix array: {(np.array(temp) + 1).tolist()}')

Next array: [-1, 0, -1, 0, 1, -1]
prefix array: [0, 1, 0, 1, 2, 0]


## 完整的匹配过程动图:

![jupyter](https://code-thinking.cdn.bcebos.com/gifs/KMP%E7%B2%BE%E8%AE%B22.gif)

## 完整代码

In [1]:
class Solution:
    def strStr(self, haystack, needle) -> int:
        if len(needle) == 0:                        # 如果 needle 为空数组, 则直接结束, 不生成前缀表
            return -1
        
        next_arr = self.get_next_arr(needle)        # 生成 next 数组
        
        j = -1           # j 用来遍历 needle
        for i in range(len(haystack)):
            while j >= 0 and haystack[i] != needle[j+1]:            # 这里的判断逻辑和 get_next_arr 里的逻辑是相同的, 这里只是把 needle[i] 换成了 haystack[i]
                j = next_arr[j] 
            
            if  haystack[i] == needle[j+1]:                         # 更新遍历参数 j,  i 在for里面更新
                j += 1                                              
            
            if j == len(needle)-1:                                  # 遍历结束的一个条件: needle 遍历到末尾
                return i - (len(needle) -1)
        
        return -1
        
        
    def get_next_arr(self, needle):
        '''
        关键思路: 使用两个指针 i,k, i指向后缀的末尾, k指向前缀的末尾
        注意: 
            1. next[index] 表示的是 index(包含index) 字符之前的 子串的 最长相等前后缀的长度 
            2. next 数组最小值是-1, 当 相同前后缀的最大长度为0时, next 数组记录的是-1 
            3. 假设needle最少得有1个元素, needle 为空字符串的情况不应生成 next 数组
        '''
        # 初始化:
        next_arr = ['' for _ in range(len(needle))]
        k = -1                              # 当 needle 只有1个元素时, 前后缀都不存在, 此时next数组应该记录为 -1
        next_arr[0] = k                    
        
        #  遍历模板串 needle, 逐步构建next数组
        for i in range(1, len(needle)):     # 注意 i 从 1 开始遍历
            
            # 处理前缀与后缀不相同的情况.  (注意: 对于当前遍历到的每一个子串, 都是从最长的前缀开始进行判断, 不相等时就缩短对应的前缀)
            # 做完这一步, 前缀末尾指针 k 要么会停在前后缀相等的位置, 要么直接为-1
            while (k > -1 and needle[i] != needle[k+1]):
                k = next_arr[k]                 # 这里按照 next 数组的定义去移动k: next[index] 表示的是 index(包含index) 字符之前的 子串的 最长相等前后缀的长度 
                                            # 前缀被缩短到前一个字符里存储的最长相等子串的位置, 这个位置在 next 数组里保存着, 由于 i 是从 needle 头部开始遍历, 所以无论如何next数组都是能找到对应值的
        
            # 处理 前后缀相等的情况.  注意: 前面的 while 走完, k 要么为-1, 要么走到前后缀相等的位置
            if needle[i] == needle[k+1]:
                k += 1                      # 如果找到了前后缀相等的位置, 那么就把 k 往右移, 用来更新 next 数组, 同时方便下一次遍历
                                            # i 的增加在 for 循环里完成

            next_arr[i] = k 
        return next_arr

In [36]:
_test = Solution()

haystack = "leetcode"; needle = "leeto"
haystack = "sadbutsad"; needle = "sad"

_test.get_next_arr(needle)
_test.strStr(haystack, needle)

0